In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
sampleTrial = pd.read_csv('sampleTrialMonth.csv').drop(['Unnamed: 0'], axis=1)
#sampleEvent = pd.read_csv('sampleEvent.csv').drop(['Unnamed: 0'], axis=1)
sampleZone = pd.read_csv('sampleZone.csv').drop(['Unnamed: 0'], axis=1)

In [109]:
eventDict = pd.read_csv('month_trafficConditions.csv').drop(['Unnamed: 0'], axis=1)['m_count']

In [110]:
eventList=[]
for i in range(len(eventDict)):
    eventList.append(pd.DataFrame.from_dict(eval(eventDict[i]), orient='index').T)

In [111]:
sampleEvent = pd.concat(eventList).reset_index().drop('index', axis=1)

In [112]:
sampleEvent = sampleEvent.reindex_axis(sorted(sampleEvent.columns), axis=1)

In [113]:
#sampleEvent = sampleEvent.drop('2015', axis=1)

In [114]:
sampleEvent.head(10)

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,2,11,0,0,0,0,0,0,0,0,0
5,0,0,4,0,3,0,0,0,0,0,0,0
6,0,1,1,0,6,0,0,0,0,0,0,0
7,0,0,1,0,2,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [115]:
m3232 = pd.DataFrame.from_csv('m3232.txt', sep="\t").reset_index()
m3232['year'] = m3232['year'].astype(str).str[:4].astype(int)

In [116]:
m3445= pd.DataFrame.from_csv('m3445.txt', sep="\t").reset_index()
m3445['year'] = m3445['year'].astype(str).str[:4].astype(int)

In [117]:
m3451 = pd.DataFrame.from_csv('m3451.txt', sep="\t").reset_index()
m3451['year'] = m3451['year'].astype(str).str[:4].astype(int)

In [118]:
#z3451 = pd.concat([z3451, pd.DataFrame(z3451.mean()).T, pd.DataFrame(z3451.mean()).T])\
#.reset_index().drop('index', axis=1)

In [119]:
#z3451['year'].iloc[6] = 2015
#z3451['year'].iloc[7] = 2016

In [120]:
m3532 = pd.DataFrame.from_csv('m3532.txt', sep="\t").reset_index()
m3532['year'] = m3532['year'].astype(str).str[:4].astype(int)

In [121]:
xx = m3451.loc[50:61]
yy = m3451.loc[50:55]

In [122]:
xx['year'] = 2015
yy['year'] = 2016

In [123]:
m3451 = pd.concat([m3451,xx,yy]).reset_index().drop('index', axis=1)

In [124]:
year = np.array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]).reshape(-1,1)
event = sampleEvent.as_matrix()

In [125]:
eventmatrix = event[:, 0:11]
event2014 = sampleEvent['2014'].as_matrix()

In [126]:
from sklearn.linear_model import Ridge

In [127]:
predict = []
error = []

for i in range(len(sampleTrial)):
    if(sampleZone['zoneid'][i]==3232):
        matrix = m3232[m3232['month']==sampleTrial['Month'][i]]
        matrix['index'] = matrix['year']-2003
        zoneMatrix  = matrix.drop(['year','month'],axis=1).set_index('index')
    
    if(sampleZone['zoneid'][i]==3445):
        matrix = m3445[m3445['month']==sampleTrial['Month'][i]]
        matrix['index'] = matrix['year']-2003
        zoneMatrix  = matrix.drop(['year','month'],axis=1).set_index('index')        
        
    if(sampleZone['zoneid'][i]==3451):
        matrix = m3451[m3451['month']==sampleTrial['Month'][i]]
        matrix['index'] = matrix['year']-2003
        zoneMatrix  = matrix.drop(['year','month'],axis=1).set_index('index')
                
    if(sampleZone['zoneid'][i]==3532):
        matrix = m3532[m3532['month']==sampleTrial['Month'][i]]
        matrix['index'] = matrix['year']-2003
        zoneMatrix  = matrix.drop(['year','month'],axis=1).set_index('index')
        
    
    a = np.zeros((14,3))
    ind = zoneMatrix.index.tolist()
    b = zoneMatrix.as_matrix()
    
    features = np.concatenate((year, np.delete(np.insert(a, ind, b, axis=0), np.add(ind, [j+1 for j in range(len(ind))]), axis=0) ), axis=1)

    xtrain = features[:-3,:]
    ytrain = eventmatrix[i]
    xtest = features[-3:-2,:]
    ytest = event2014[i]
    
    #if(np.count_nonzero(ytrain) > 5):
        #clf = Ridge(alpha=15)
    #else:
    clf = Ridge(alpha=30)
    
    clf.fit(xtrain, ytrain)    
    predict.append(clf.predict(xtest)[0])
    error.append(((clf.predict(xtest)[0] - ytest) ** 2))
    

In [36]:
errA = np.sqrt(np.sum(error)/len(error))
errA

19.704378177265244

In [61]:
errD = np.sqrt(np.sum(error)/len(error))
errD

11.149289915477009

In [81]:
errO = np.sqrt(np.sum(error)/len(error))
errO

4.1376977513049216

In [387]:
errP = np.sqrt(np.sum(error)/len(error))
errP

0.17648032380147063

In [105]:
errR = np.sqrt(np.sum(error)/len(error))
errR

3.4689826856240726

In [128]:
errT = np.sqrt(np.sum(error)/len(error))
errT

0.6625205064847457

In [131]:
np.sqrt((errA**2 + errD**2 + errO**2 + 0.1748**2 + errR**2 + errT**2)/6)

9.5060775032596059